In [1]:
from py2neo import Graph, Node, Relationship
from pandas import DataFrame
#import json
#json.dumps(list)

In [2]:
graph = Graph("neo4j+s://44bb2475.databases.neo4j.io", auth=("neo4j", "n04yHsQNfrl_f72g79zqMO8xVU2UvUsNJsafcZMtCFM"))

In [4]:
dtf_data = DataFrame([dict(_) for _ in graph.run("MATCH (u:user) RETURN DISTINCT u.fname, u.lname, u.email")])
dtf_data

,u.fname,u.lname,u.email
0,Hari,Krish,hk@lbl.gov
1,John,Doe,jdoe@yahoo.com
2,Howard,Yanxon,yanxonh@anl.gov
3,Alex,Hexemer,ahexemer@lbl.gov


Confirmed Content Assets: (asset doesn't exist LOL)
(1) Workflow
(2) Model
(3) 

In [23]:
### SERVICE_API ###
def add_user(fname:str, lname:str, email:str, role:str):
    dbindx = len([dict(_) for _ in graph.run("MATCH (u:user) RETURN u")]) + 1
    temp_id = str(fname[0] + lname + str(dbindx).zfill(5))
    cypher_query = '''
    MATCH (r:Role {name:$role})
    CREATE (n:Subject:user:Primitive {uid:$temp_id, fname:$fname, lname:$lname, email:$email})-[:has_attr]->(r)
    '''
    graph.run(cypher_query, parameters = {'temp_id':temp_id, 'fname':fname, 'lname':lname, 'email':email, 'role':role})

def add_compute_loc(cname:str, cloc:str):
    cypher_query = '''
    CREATE(n:compute:Object:Location {cname:$cname, cloc:$cloc})
    '''
    graph.run(cypher_query, parameters = {'cname':cname, 'cloc':cloc})

def add_content_asset(aname:str, atype:str, owner:str):
    auid = str(owner + '_')
    cypher_query = '''
    CREATE (n:content:Object:Primitive {aname:$aname, atype:$atype, a_uid:toString(($auid)+toString(timestamp())), owner:$owner})
    RETURN n.a_uid
    '''
    a_uid=graph.run(cypher_query, parameters = {'aname':aname, 'atype':atype, 'auid':auid, 'owner':owner}).evaluate()
    cypher_query= '''
    MATCH (u:user {uid:$owner})
    MATCH (c:content {a_uid:$a_uid})
    MERGE (u)-[rel:owner_of]->(c)
    '''
    graph.run(cypher_query, parameters = {'a_uid':a_uid, 'owner':owner})

def add_user_asset(uaname:str, uatype:str, ua_srcpath:str, owner:str):
    uauid = str(owner + '_')
    cypher_query='''
    CREATE (n:user_asset:Object:Primitive {uaname:$uaname, uatype:$uatype, ua_srcpath:$ua_srcpath, ua_uid:toString(($uauid)+toString(timestamp())), owner:$owner})
    RETURN n.ua_uid
    '''
    ua_uid=graph.run(cypher_query, parameters = {'uaname':uaname, 'uatype':uatype, 'ua_srcpath':ua_srcpath, 'uauid':uauid, 'owner':owner}).evaluate()
    cypher_query= '''
    MATCH (u:user {uid:$owner})
    MATCH (ua:user_asset {ua_uid:$ua_uid})
    MERGE (u)-[rel:owner_of]->(ua)
    '''
    graph.run(cypher_query, parameters = {'owner':owner,'ua_uid':ua_uid})

def delete_user(uid:str):
    cypher_query = '''
    MATCH (u:user {uid:$uid})
    DETACH DELETE (u)
    '''
    graph.run(cypher_query, parameters = {'uid':uid})

def delete_compute_loc(cname:str, cloc:str):
    cypher_query = '''
    MATCH (c:compute:Object:Location {cname:$cname, cloc:$cloc})
    DETACH DELETE (c)
    '''
    graph.run(cypher_query, parameters = {'cname':cname, 'cloc':cloc})

def delete_content_asset(aname:str):
    # deletes an asset node and all of its relationships
    cypher_query = '''
    MATCH (c:content {aname:$aname})
    DETACH DELETE (c)
    '''
    graph.run(cypher_query, parameters = {'aname':aname})

def delete_user_asset(uaname:str):
    # deletes an asset node and all of its relationships
    cypher_query = '''
    MATCH (ua:user_asset {uaname:$uaname})
    DETACH DELETE (ua)
    '''
    graph.run(cypher_query, parameters = {'uaname':uaname})

### MANAGE TEAM  -- USES 'TEAM' 'ATTRIBUTE' PROPERTIES ###

def add_user_team(tname:str, creator:str, target_uid:str):
    cypher_query = '''
    MATCH (u:user {uid:$target_uid}), (t:Team:Attribute {name:$tname, creator:$creator})
    MERGE (u)-[:has_attr]->(t)
    '''
    graph.run(cypher_query, parameters = {'uid':target_uid, 'tname':tname, 'creator':creator})

def create_user_team(tname:str, tlist:list, creator:str):
    # takes a list of user nodes and creates a label and applies label to user nodes
    # team names must be unique per creator
    cypher_query='''
    CREATE (team:Team:Attribute {name:$tname, creator:$creator})
    '''
    graph.run(cypher_query, parameters ={'tname':tname, 'creator':creator})

    for uid in tlist:
        cypher_query='''
        MATCH (u:user {uid:$uid}), (t:Team:Attribute {name:$tname, creator:$creator})
        MERGE (u)-[:has_attr]->(t)
        '''
        graph.run(cypher_query, parameters = {'uid':uid, 'tname':tname, 'creator':creator})

def delete_user_team(tname:str, creator:str, target_uid:str):
    # takes a list of user nodes and creates a label and applies label to user nodes
    # team names must be unique per creator
    cypher_query='''
    MATCH (n:Team:Attribute {name:$tname, creator:$creator})
    DETACH DELETE (n)
    '''
    graph.run(cypher_query, parameters ={'tname':tname, 'creator':creator})

def remove_user_team(tname:str, creator:str, target_uid:str):
    cypher_query = '''
    MATCH (u:user {uid:$target_uid}) -[rel:has_attr]-> (t:Team:Attribute {name:$tname, creator:$creator})
    DELETE rel
    '''
    graph.run(cypher_query, parameters = {'uid':target_uid, 'tname':tname, 'creator':creator})

### MANAGE ROLES -- USES 'ROLE' 'ATTRIBUTE' PROPERTIES ###
def assign_user_role(uid:str, rname:str):
    # assign a user by uid to a role by role_id
    cypher_query = '''
    MATCH (u:user {uid:$uid}), (r:Role {name:$rname})
    MERGE (u)-[:has_attr]->(r)
    '''
    graph.run(cypher_query, parameters = {'uid':uid, 'rname':rname})

def create_role(rname:str):
    # create a new role
    cypher_query = '''
    CREATE(n:Role:Attribute {name:$rname})
    '''
    graph.run(cypher_query, parameters = {'rname':rname})

def delete_role(rname:str):
    # delete existing role
    cypher_query = '''
    MATCH (r:Role {name:$rname})
    DETACH DELETE (r)
    '''
    graph.run(cypher_query, parameters = {'rname':rname})

def remove_user_role(target_uid:str, rname:str):
    # remove role assignment from a unique user
    cypher_query = '''
    MATCH (u:user {uid:$target_uid})-[rel:has_attr]->(r:Role {name:$rname})
    DELETE rel
    '''
    graph.run(cypher_query, parameters = {'target_uid':target_uid, 'rname':rname})

### CREATE ACTIONS ###
def create_actions_basic():
    cypher_query = '''
    MERGE (read:Action {name:'Read'})-[:has_attr]->(fullAccess:Attribute:Group
    {name: 'Full Access'})<-[:has_attr]-(write:Action {name:'Write'})
    '''
    graph.run(cypher_query)

### FETCH INFORMATION ###
def get_user_info(uid:str):
    cypher_query = '''
    MATCH (u:user) WHERE u.uid = $uid
    RETURN DISTINCT u.fname as Firstname, u.lname as Lastname, u.email as Email
    '''
    output = DataFrame([dict(_) for _ in graph.run(cypher_query, parameters = {'uid':uid})])
    return output


In [13]:
### MANAGE ASSET ACCESS ###
def assign_asset_manager(aname:str, target_uid:str):
    # gives read/write/share privileges
    cypher_query = '''
    '''
    graph.run(cypher_query, parameters = {'target_uid':target_uid, 'aname':aname})

def share_asset_read_group(aname:str, gid:str, pname:str):
    # gives read-only privileges
    cypher_query = '''
    MATCH (sub:Attribute {gid:$gid}), (obj:Attribute {name:$aname}), (act:Attribute {name:"Read"})
    CREATE (pol:Policy {name:$pname, pid:toString($aname + $pname), decision:"Permit"})
    MERGE (pol)<-[:SUB_CON]-(sub)
    MERGE (pol)<-[:OBJ_CON]-(obj)
    MERGE (pol)<-[:ACT_CON]-(act)
    '''
    graph.run(cypher_query, parameters = {'gid':gid, 'aname':aname, 'pname':pname})

def share_asset_read_user(aname:str, target_uid:str):
    # gives read-only privileges
    cypher_query = '''
    MATCH (u:user {uid:$target_uid}), (a:asset {aname:$aname})
    CREATE (u) -[rel:has_attr]-> (a)
    '''
    graph.run(cypher_query, parameters = {'target_uid':target_uid, 'aname':aname})

def share_asset_write_group(aname:str, gid:str):
    # gives read/write privileges
    cypher_query = '''
    MATCH (sub:Attribute {gid:$gid}), (obj:Attribute {name:$aname}), (act:Attribute {name:"Write"})
    CREATE (pol:Policy {name:$pname, pid:toString(($pname)+toString(timestamp())), decision:"Permit"})
    MERGE (pol)<-[:SUB_CON]-(sub)
    MERGE (pol)<-[:OBJ_CON]-(obj)
    MERGE (pol)<-[:ACT_CON]-(act)
    '''
    graph.run(cypher_query, parameters = {'target_uid':target_uid, 'aname':aname})

def share_asset_write_user(aname:str, target_uid:str):
    # gives read/write privileges
    cypher_query = '''
    MATCH (u:user {uid:$target_uid}), (a:asset {aname:$aname})
    CREATE (u) -[rel:has_attr]-> (a)
    '''
    graph.run(cypher_query, parameters = {'target_uid':target_uid, 'aname':aname})

#def remove_asset_access_user(aname:str, uid:str):
#def remove_asset_access_group(aname:str, gid:str):
#def request_asset_access():
#def request_compute_resources():

Use the code below to generate a sample graph upon which to run policies. Please note the 

In [20]:
#create action nodes for policy reference
create_actions_basic()

# create three new roles
create_role('admin')
create_role('subadmin')
create_role('general user')

# add five users with different roles to database
add_user("Hari","Krish", "hk@lbl.gov", 'admin')
add_user("John", "Doe", "jdoe@yahoo.com", 'general user')
add_user("Howard", "Yanxon", "yanxonh@anl.gov", 'subadmin')
add_user("Alex", "Hexemer", "ahexemer@lbl.gov", 'admin')
add_user("Jane", "Dough", "jdough@gmail.com", 'general user')

In [21]:
# add one asset to content registry
add_content_asset("trainedmodel_01", "trained_model", "JDoe00002")

# add a private user asset (dataset local to user)
add_user_asset("private_data", "dataset", "data/cynanobacteria.npy", 'JDough00005')

In [22]:
# create a team of existing users
ulist=['AHexemer00004','JDough00005','JDoe00002']
create_user_team('team01', ulist, 'HKrish00001')


Below this Markdown box are incomplete definitions that will likely require the creation of policies.

In [116]:
# share file asset in three different ways
assign_asset_manager("trainedmodel_01", "HKrish00001")

In [113]:
delete_content_asset('trainedmodel_01')

In [112]:
delete_user_asset('private_data')

In [41]:
delete_user('JDough00005')

In [108]:
remove_user_role('AHexemer00004', 'admin')

In [11]:
ulist=['AHexemer00004','JDough00005','JDoe00002']

In [ ]:
delete_user_team